In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import numpy as np
import os
from pathlib import Path
from pyteomics import mass as pymass

In [3]:
first = pd.read_csv("/home/robbe/ionbot/full_projects/PXD006657_first.csv")
first.head()

,Unnamed: 0,ionbot_match_id,spectrum_title,scan,spectrum_file,precursor_mass,peptide_mass,observed_retention_time,charge,database_peptide,...,Organism,Cell line,Oxidant treatment,Oxidant dose (µM),Secondary treatment,Secondary treatment dose (µM),Length of oxidant treatment (mins),Length of Secondary treatment (mins),Notes,First/Lower/Coeluting
0,0,0_3960_1,controllerType=0 controllerNumber=1 scan=10045,10045,A0288_klc_20140708m_A431_Redoxome_H2O2_BR1_C1,2410.12550,21519.7893,2308.64970,4,TKPAGPCPGCEHSQESAQIR,...,Human,A431,H2O2,20,None,0,10,0,"Treated was light labeled, non-treated was hea...",First
1,1,0_3967_4,controllerType=0 controllerNumber=1 scan=10053,10053,A0288_klc_20140708m_A431_Redoxome_H2O2_BR1_C1,1728.84170,17318.8719,2309.49618,3,VRMRPGVACSVSQAQK,...,Human,A431,H2O2,20,None,0,10,0,"Treated was light labeled, non-treated was hea...",First
2,2,0_3974_1,controllerType=0 controllerNumber=1 scan=10060,10060,A0288_klc_20140708m_A431_Redoxome_H2O2_BR1_C1,2067.97463,18098.3154,2310.06582,3,GCNPSGHTQSVTTPEPAK,...,Human,A431,H2O2,20,None,0,10,0,"Treated was light labeled, non-treated was hea...",First
3,3,0_3982_1,controllerType=0 controllerNumber=1 scan=10069,10069,A0288_klc_20140708m_A431_Redoxome_H2O2_BR1_C1,2330.03632,20718.9386,2310.99708,3,KGEDEDKGPPCGPVNCNEK,...,Human,A431,H2O2,20,None,0,10,0,"Treated was light labeled, non-treated was hea...",First
4,4,0_3992_2,controllerType=0 controllerNumber=1 scan=10080,10080,A0288_klc_20140708m_A431_Redoxome_H2O2_BR1_C1,1734.86092,17318.8719,2312.07936,3,VRMRPGVACSVSQAQK,...,Human,A431,H2O2,20,None,0,10,0,"Treated was light labeled, non-treated was hea...",First


In [4]:
first.columns

Index(['Unnamed: 0', 'ionbot_match_id', 'spectrum_title', 'scan',
       'spectrum_file', 'precursor_mass', 'peptide_mass',
       'observed_retention_time', 'charge', 'database_peptide',
       'matched_peptide', 'modifications', 'modifications_delta',
       'corrected_retention_time', 'unexpected_modification', 'database',
       'psm_score', 'q-value', 'PEP', 'proteins', 'Accession', 'Organism',
       'Cell line', 'Oxidant treatment', 'Oxidant dose (µM)',
       'Secondary treatment', 'Secondary treatment dose (µM)',
       'Length of oxidant treatment (mins)',
       'Length of Secondary treatment (mins)', 'Notes',
       'First/Lower/Coeluting'],
      dtype='object')

In [7]:
first[['unexpected_modification', 'modifications_delta']].head()

,unexpected_modification,modifications_delta
0,[357]probiotinhydrazide[P],10|57.0215|6|258.1150
1,NaN,3|15.9949
2,[357]probiotinhydrazide[P],4|258.1150
3,[357]probiotinhydrazide[P],11|57.0215|13|258.1150
4,NaN,3|15.9949


In [15]:
deltas = first['modifications_delta']
deltas.head()

0     10|57.0215|6|258.1150
1                 3|15.9949
2                4|258.1150
3    11|57.0215|13|258.1150
4                 3|15.9949
Name: modifications_delta, dtype: object

In [18]:
first['deltamass'] = first['modifications_delta'].str.split("|")
first['deltamass'] = first['deltamass'].str[1::2]
explodedPSMs = first.explode('deltamass')
deltamasscounts = explodedPSMs['deltamass'].value_counts().rename_axis('deltamass').reset_index(name='counts')
deltamasscounts

,deltamass,counts
0,,49061
1,258.1150,18158
2,15.9949,15345
3,57.0215,15314
4,195.0756,2276
...,...,...
377,327.2410,1
378,34.0631,1
379,310.0477,1
380,44.9851,1
